# Question Answering Model 

- dataset
- torch
- transformers
- transformers[torch]
- evaluate

import packages

In [1]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    DefaultDataCollator,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer
)
import torch

import evaluate
import collections
from tqdm.auto import tqdm
import numpy as np

import os
import re
import datetime

/mount/studenten-temp1/users/linku/AL_QA/.venv/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Set cache directory.

In [2]:
CACHE_DIR=os.path.abspath(os.path.expanduser('cache'))
%set_env TRANSFORMERS_CACHE $CACHE_DIR
%set_env HF_MODULES_CACHE $CACHE_DIR
%set_env HF_DATASETS_CACHE $CACHE_DIR

env: TRANSFORMERS_CACHE=/mount/studenten-temp1/users/linku/AL_QA/cache
env: HF_MODULES_CACHE=/mount/studenten-temp1/users/linku/AL_QA/cache
env: HF_DATASETS_CACHE=/mount/studenten-temp1/users/linku/AL_QA/cache


### arguments.py

args_input.

In [3]:
args_input_ALstrategy = 'MarginSampling'
args_input_initseed = 100 # 1000
args_input_quota = 100 # 1000
args_input_batch = 35 # 128
args_input_dataset_name = 'SQuAD'
args_input_iteration = 1
args_input_model_batch = 8 # already add in arguments.py

### load dataset

In [4]:
squad = load_dataset(args_input_dataset_name.lower())
# squad["train"] = squad["train"].shuffle(42).select(range(2000))
squad["train"] = squad["train"].select(range(3000))
squad["validation"] = squad["validation"].select(range(1000))

Found cached dataset squad (/home/users1/linku/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00,  2.20it/s]


Next we will preprocess the dataset (training and evaluation data).

In [5]:
def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs["offset_mapping"]
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []
    example_ids = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)
        
        example_ids.append(examples["id"][sample_idx]) # newly added for used in unlabel data predict

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["example_id"] = example_ids
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [6]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [7]:
max_length = 384
stride = 128

# load tokenizer for dataset preprocessing
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# preprocess data
train_dataset = squad["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=squad["train"].column_names,
)
val_dataset = squad["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=squad["validation"].column_names,
)

Loading cached processed dataset at /home/users1/linku/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-2a9d31b0890526df.arrow
Loading cached processed dataset at /home/users1/linku/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-0f087a92e258c27b.arrow


In [8]:
len(train_dataset['example_id']) # 10138 more than before tokenizing

3074

In [9]:
len(set(train_dataset['example_id'])) # 10000 same as before tokenizing

3000

## Evaluation

In [10]:
metric = evaluate.load("squad")

In [11]:
def compute_metrics(start_logits, end_logits, val_dataset, examples):
    
    example_to_features = collections.defaultdict(list)
    max_answer_length = 30
    n_best = 20
    for idx, feature in enumerate(val_dataset):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = val_dataset[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [64]:
def get_prob(start_logits, end_logits, features, examples):
    # thinking the posibility of get rid of 'examples'
    prob_list_dict = []
    example_to_features = collections.defaultdict(list)
    max_answer_length = 30
    n_best = 20 # TODO: if set n_best as 5, will it effect the time??
    
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    for example in tqdm(examples):
        example_id = example["id"]
        # context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answers.append(start_logit[start_index] + end_logit[end_index])
        
        if len(answers) > 1:
            answers.sort(reverse=True)
            n_best_sorted_answers = answers[:n_best]

            n_best_sorted_prob = softmax(n_best_sorted_answers)
            prob_list_dict.append(
                {'idx': example_to_features[example_id], 
                 'probs': n_best_sorted_prob}
            )

        elif example_to_features[example_id]:
            prob_list_dict.append(
                {'idx': example_to_features[example_id], 
                 'probs': np.array([0])}
            )
    
    return prob_list_dict
# move to evaluation.py

## utils.py

In [94]:
def get_preds(trainer_qs, unlabeled_data):
	preds, _, _ = trainer_qs.predict(unlabeled_data)
	start_logits, end_logits = preds
	return start_logits, end_logits

# move to utils.py

In [95]:
def get_unlabel_data(n_pool, labeled_idxs, train_dataset):
    unlabeled_idxs = np.arange(n_pool)[~labeled_idxs]
    unlabeled_data = train_dataset.select(indices=unlabeled_idxs)
    return unlabeled_data

# move to utils.py

In [96]:
def softmax(x): 
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)
# move to utils.py

## Query

In [12]:
def random_sampling_query(labeled_idxs):
    return np.random.choice(np.where(labeled_idxs==0)[0], NUM_QUERY, replace=False)

In [13]:
def margin_sampling_query(n_pool, labeled_idxs, train_dataset, trainer_qs, example):
    
    unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    start_logits, end_logits, = get_preds(trainer_qs, unlabeled_data)
    prob_list_dict = get_prob(start_logits, end_logits, unlabeled_data, example)

    uncertainties_list_dict = []
    for d in prob_list_dict:
        if len(d['probs']) > 1: # if prob_dict['probs'] is not 0
            uncertainties = d['probs'][0] - d['probs'][1]
            uncertainties_list_dict.append(
                {'idx': d['idx'], 
                 'uncertainties': uncertainties}
                 )
        elif d['idx']:
            uncertainties_list_dict.append(
                {'idx': d['idx'], 
                 'uncertainties': np.array([0])}
                 )
    
    sorted_uncertainties_dict = sorted(uncertainties_list_dict, key=lambda d: d['uncertainties']) # TODO: check smallest or largest, now is smallest    
    return [uncertainties_dict['idx'][0] for uncertainties_dict in sorted_uncertainties_dict[:NUM_QUERY]]

In [97]:
def least_confidence_query(n_pool, labeled_idxs, train_dataset, trainer_qs, example):

    unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    start_logits, end_logits, = get_preds(trainer_qs, unlabeled_data)
    probs_list_dict = get_prob(start_logits, end_logits, unlabeled_data, example)

    confidence_list_dict = []
    for d in probs_list_dict:
        if len(d['probs']) > 1: # if prob_dict['probs'] is not 0
            # uncertainties_d = d['probs']
            confidence = max(d['probs'])
            confidence_list_dict.append(
                {'idx': d['idx'], 
                    'confidence': confidence}
                    )
        elif d['idx']:
            confidence_list_dict.append(
                {'idx': d['idx'], 
                    'confidence': np.array([0])}
                    )

    sorted_confidence_dict = sorted(confidence_list_dict, key=lambda d: d['confidence'])
    return [confidence_dict['idx'][0] for confidence_dict in sorted_confidence_dict[:NUM_QUERY]]

In [98]:
def var_ratio_query(n_pool, labeled_idxs, train_dataset, trainer_qs, example):

    unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    start_logits, end_logits, = get_preds(trainer_qs, unlabeled_data)
    probs_list_dict = get_prob(start_logits, end_logits, unlabeled_data, example)

    confidence_list_dict = []
    for d in probs_list_dict:
        if len(d['probs']) > 1: # if prob_dict['probs'] is not 0
            # uncertainties_d = d['probs']
            confidence = max(d['probs'])
            confidence_list_dict.append(
                {'idx': d['idx'], 
                    'confidence': confidence}
                    )
        elif d['idx']:
            confidence_list_dict.append(
                {'idx': d['idx'], 
                    'confidence': np.array([0])}
                    )

    sorted_confidence_dict = sorted(confidence_list_dict, key=lambda d: d['confidence'])
    return [confidence_dict['idx'][0] for confidence_dict in sorted_confidence_dict[:NUM_QUERY]]

# main.py

### parameters

In [14]:
NUM_QUERY = args_input_batch
NUM_INIT_LB = args_input_initseed
NUM_ROUND = int(args_input_quota / args_input_batch)
DATA_NAME = args_input_dataset_name
STRATEGY_NAME = args_input_ALstrategy

### seed and device

In [15]:
SEED = 4666
# os.environ['TORCH_HOME']='./basicmodel'
os.environ["CUDA_VISIBLE_DEVICES"] = str(3)

# fix random seed
np.random.seed(SEED)
torch.manual_seed(SEED)
# torch.backends.cudnn.enabled  = True
# torch.backends.cudnn.benchmark= True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### start experiment

In [16]:
iteration = args_input_iteration
model_batch = args_input_model_batch

all_acc = []
acq_time = []

In [ ]:
# repeate # iteration trials
while (iteration > 0): 
	iteration = iteration - 1

	## data, network, strategy
	net = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased").to(device)

	## set up training
	training_args = TrainingArguments(
		output_dir="./models",
		evaluation_strategy="no",
		eval_steps=100,
		logging_steps=100,
		learning_rate=1e-4,
		per_device_train_batch_size=model_batch,
		per_device_eval_batch_size=model_batch, 
		gradient_accumulation_steps=1,
		num_train_epochs=3,  # max_steps will override this value
		# max_steps=1000,  # comment out if this is not wanted
		weight_decay=0.01,
		report_to="none",
	)

	## data collator for batching
	data_collator = DefaultDataCollator()

	start = datetime.datetime.now()

	## generate initial labeled pool
	n_pool = len(train_dataset)
	labeled_idxs = np.zeros(n_pool, dtype=bool)

	tmp_idxs = np.arange(n_pool)
	np.random.shuffle(tmp_idxs)
	labeled_idxs[tmp_idxs[:args_input_initseed]] = True

	run_0_labeled_idxs = np.arange(n_pool)[labeled_idxs] 
	
	## record acc performance 
	acc = np.zeros(NUM_ROUND + 1) # build 3 runs + run_0 # origin 10 runs + run_0

	## the actual trainer which performs training and evaluation
	trainer_0 = Trainer(
						model=net,
						args=training_args,
						train_dataset=train_dataset.select(indices=run_0_labeled_idxs),
						eval_dataset=val_dataset,
						tokenizer=tokenizer,
						data_collator=data_collator
						)	
		
	## print info
	print(DATA_NAME)
	# print('RANDOM SEED {}'.format(SEED))
	print(STRATEGY_NAME) # print(type(strategy).__name__)
	
	## round 0 accuracy 
	trainer_0.train() # already update to main.py
	model_saved_dir = 'models/train_bert_squad_0'
	trainer_0.save_model(model_saved_dir)

	preds, _, _ = trainer_0.predict(val_dataset)
	start_logits, end_logits = preds
	acc[0] = compute_metrics(start_logits, end_logits, val_dataset, squad["validation"])['exact_match']

	trainer_qs = trainer_0

	# print('Round 0\ntesting accuracy {}'.format(acc[0]))
	# print('\n')
	
	# ## round 1 to rd
	# for rd in range(1, NUM_ROUND+1):
	# 	print('Round {}'.format(rd))
		
	# 	## query
	# 	if STRATEGY_NAME == 'RandomSampling':
	# 		q_idxs = random_sampling_query(labeled_idxs)
	# 	elif STRATEGY_NAME == 'MarginSampling':
	# 		q_idxs = margin_sampling_query(n_pool, labeled_idxs, train_dataset, trainer_qs, squad['train'])
	# 	elif STRATEGY_NAME == 'LeastConfidence':
	# 		q_idxs = least_confidence_query(n_pool, labeled_idxs, train_dataset, trainer_qs, squad['train'])
	# 	elif STRATEGY_NAME == 'EntropySampling':
	# 		q_idxs = entropy_query()
	# 	elif STRATEGY_NAME == 'MarginSamplingDropout':
	# 		q_idxs = margin_sampling_dropout_query()
	# 	elif STRATEGY_NAME == 'LeastConfidenceDropout':
	# 		q_idxs = least_confidence_dropout_query()
	# 	elif STRATEGY_NAME == 'EntropySamplingDropout':
	# 		q_idxs = entropy_dropout_query()
	# 	elif STRATEGY_NAME == 'VarRatio':
	# 		q_idxs = var_ratio_query(n_pool, labeled_idxs, train_dataset, trainer_qs, squad['train'])
	# 	elif STRATEGY_NAME == 'KMeansSampling':
	# 		q_idxs = kmeans_query()
	# 	elif STRATEGY_NAME == 'KCenterGreedy':
	# 		q_idxs = kcenter_query()
	# 	# elif STRATEGY_NAME == 'KCenterGreedyPCA': # not sure
	# 	# 	q_idxs = 
	# 	elif STRATEGY_NAME == 'BALDDropout':
	# 		q_idxs = bayesian_query()
	# 	elif STRATEGY_NAME == 'MeanSTD':
	# 		q_idxs = mean_std_query()
	# 	elif STRATEGY_NAME == 'BadgeSampling':
	# 		q_idxs = badge_query()
	# 	elif STRATEGY_NAME == 'LossPredictionLoss':
	# 		# different net!
	# 		q_idxs = loss_prediction_query()
	# 	elif STRATEGY_NAME == 'CEALSampling':
	# 		# why use 'CEALSampling' in STRATEGY_NAME
	# 		q_idxs = ceal_query()
	# 	else:
	# 		raise NotImplementedError
	
	# 	## update
	# 	labeled_idxs[q_idxs] = True
	# 	run_rd_labeled_idxs = np.arange(n_pool)[labeled_idxs]
	# 	print('run_rd_labeled_idxs', len(run_rd_labeled_idxs))

	# 	trainer_rd = Trainer(
	# 					model=AutoModelForQuestionAnswering.from_pretrained(model_saved_dir).to(device),
	# 					args=training_args,
	# 					train_dataset=train_dataset.select(indices=run_rd_labeled_idxs),
	# 					eval_dataset=val_dataset,
	# 					tokenizer=tokenizer,
	# 					data_collator=data_collator
	# 					)

	# 	## train # already update to main.py
	# 	trainer_rd.train()
	# 	model_saved_dir = 'train_bert_squad_' + rd
	# 	trainer_rd.save_model(model_saved_dir)

	# 	trainer_qs = trainer_rd
	
	# 	## round rd accuracy
	# 	preds, _, _ = trainer_rd.predict(val_dataset)
	# 	start_logits, end_logits = preds
	# 	acc[rd] = compute_metrics(start_logits, end_logits, val_dataset, squad["validation"])['exact_match']
	# 	print('testing accuracy {}'.format(acc[rd]))
	# 	print('\n')

	# 	torch.cuda.empty_cache()
	
	# ## print results
	# print('SEED {}'.format(SEED))
	# print(STRATEGY_NAME)
	# print(acc)
	# all_acc.append(acc)
	
	# ## save model
	# timestamp = re.sub('\.[0-9]*','_',str(datetime.datetime.now())).replace(" ", "_").replace("-", "").replace(":","")
	# model_path = './modelpara/' + timestamp + DATA_NAME+ '_'  + STRATEGY_NAME + '_' + str(NUM_QUERY) + '_' + str(NUM_INIT_LB) +  '_' + str(args_input_quota)  +'.params'
	# end = datetime.datetime.now()
	# acq_time.append(round(float((end - start).seconds), 3))


## query workspace

In [18]:
trainer_qs = trainer_0

In [20]:
# get unlable data
unlabeled_idxs = np.arange(n_pool)[~labeled_idxs]
unlabeled_data = train_dataset.select(indices=unlabeled_idxs)
len(unlabeled_idxs)

In [22]:
# smaller data
unlabeled_idxs_20 = unlabeled_idxs[:20]
unlabeled_data_20 = train_dataset.select(unlabeled_idxs_20)
len(unlabeled_data_20)

20

In [66]:
preds_20, _, _ = trainer_qs.predict(unlabeled_data_20)
start_logits_20, end_logits_20 = preds_20
# query_index_20 = query(start_logits_20, end_logits_20, unlabeled_data_20, squad['train'])
# query_index_20
# start = datetime.datetime.now()
probs_list_dict_20 = get_prob(start_logits_20, end_logits_20, unlabeled_data_20, squad['train'])
# end = datetime.datetime.now()
# print(end - start)
probs_list_dict_20

100%|██████████| 3000/3000 [00:00<00:00, 5616.51it/s]


[{'idx': [0],
  'probs': array([0.11115894, 0.09532093, 0.0930382 , 0.0610937 , 0.05865714,
         0.05302327, 0.04550183, 0.04496831, 0.04450177, 0.04440406,
         0.0430655 , 0.04062211, 0.03973364, 0.03954602, 0.03903756,
         0.03536009, 0.02944322, 0.02852237, 0.02751693, 0.02548442],
        dtype=float32)},
 {'idx': [1],
  'probs': array([0.12108769, 0.07055194, 0.06747304, 0.0624703 , 0.058971  ,
         0.05589368, 0.05567102, 0.05070628, 0.04621866, 0.0458475 ,
         0.04385964, 0.04292731, 0.04140681, 0.0389948 , 0.03859771,
         0.03560026, 0.03373416, 0.03249145, 0.03064777, 0.02684893],
        dtype=float32)},
 {'idx': [2],
  'probs': array([0.09742153, 0.08271339, 0.07118762, 0.06648113, 0.06342224,
         0.06291201, 0.05753821, 0.0547342 , 0.05338069, 0.05239553,
         0.04952316, 0.03969455, 0.03712869, 0.03346546, 0.03322176,
         0.03211612, 0.02906213, 0.02878195, 0.02755215, 0.02726741],
        dtype=float32)},
 {'idx': [3],
  'probs': 

In [70]:
q_idxs = get_margin(probs_list_dict_20, 5)
q_idxs

[12, 2, 0, 14, 13]

In [87]:
def get_least_conf(probs_list_dict, n):
    confidence_list_dict = []
    for d in probs_list_dict:
        if len(d['probs']) > 1: # if prob_dict['probs'] is not 0
            # uncertainties_d = d['probs']
            confidence = max(d['probs'])
            confidence_list_dict.append(
                {'idx': d['idx'], 
                    'confidence': confidence}
                    )
        elif d['idx']:
            confidence_list_dict.append(
                {'idx': d['idx'], 
                    'confidence': np.array([0])}
                    )

    sorted_confidence_dict = sorted(confidence_list_dict, key=lambda d: d['confidence'])
    return [confidence_dict['idx'][0] for confidence_dict in sorted_confidence_dict[:n]]

In [88]:
q_idxs = get_least_conf(probs_list_dict_20, 5)
q_idxs

[2, 0, 4, 1, 3]

In [89]:
# predict with unlable data
preds, _, _ = trainer_qs.predict(unlabeled_data)

In [91]:
start_logits, end_logits = preds
probs_list_dict = get_prob(start_logits, end_logits, unlabeled_data, squad['train'])

100%|██████████| 3000/3000 [00:16<00:00, 185.87it/s]


In [ ]:
q_idxs = get_margin(probs_list_dict, NUM_QUERY)
q_idxs

In [ ]:
q_idxs = get_least_conf(probs_list_dict, NUM_QUERY)
q_idxs